In [1]:
import csv
from nltk.tokenize import word_tokenize, sent_tokenize
import string
import numpy as np

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# Initialization
rng = np.random

# Parameters
learning_rate = 0.01
training_epochs = 25
display_step = 10

# Network Parameters
batch_size = 100
vocab_size = 10000
n_hidden_1 = 256
n_hidden_2 = 256
n_embedding = 256
positive_samples = 4

data_index = 0
window_size = 25
half_window = round((window_size - 1) / 2)
num_skips = 2
words_per_batch = round(batch_size / num_skips)


# Model itself
layer_input = tf.placeholder("float", [batch_size, vocab_size], "layer_input")
layer_output_nums = tf.placeholder(tf.int32, [batch_size, positive_samples], "layer_output")

w_l1 = tf.Variable(tf.random_normal([vocab_size, n_hidden_1]))
w_l1_bias = tf.Variable(tf.random_normal([n_hidden_1]))
layer_1_sum = tf.add(tf.matmul(layer_input, w_l1), w_l1_bias)
layer_1 = tf.nn.relu(layer_1_sum)


w_l2 = tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]))
w_l2_bias = tf.Variable(tf.random_normal([n_hidden_2]))
layer_2_sum = tf.add(tf.matmul(layer_1, w_l2), w_l2_bias)

w_out = tf.Variable(tf.random_normal([vocab_size, n_hidden_2]))
w_out_bias = tf.Variable(tf.random_normal([vocab_size]))

loss = tf.nn.sampled_softmax_loss(w_out, w_out_bias, layer_output_nums, layer_2_sum, 
                                  num_sampled = batch_size, num_classes = vocab_size, num_true = positive_samples)
cost = tf.reduce_mean(loss)
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

init = tf.global_variables_initializer()
train_inputs = tf.placeholder(tf.float64, shape=[batch_size, vocab_size], name = "train_inputs")

c:\users\jvret\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\jvret\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\jvret\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\jvret\appd

Instructions for updating:
non-resource variables are not supported in the long term


In [16]:
import nltk
nltk.download('punkt')

# with open('train.pair_tok.tsv', encoding='utf-8') as f:
#     sentences = csv.reader(f)
#     words=[]
#     for sentence in sentences:
#         tokenized_sents = [word_tokenize(i) for i in sentence]
#         for token in tokenized_sents:
#             for each in token:
#                 lower_text = each.lower()
#                 punc = string.punctuation
#                 punc = punc.replace('-','')
#                 punc = punc.replace('\'','')
#                 punc = list(punc)
#                 for p in punc:
#                     lower_text = lower_text.replace(p,' ')
#                 lower_text = lower_text.replace('-',' ')
#                 words.append(lower_text)

def get_words(file_loc):
    with open(file_loc, encoding='utf-8') as f:
        sentences = csv.reader(f)
        words=[]
        for sentence in sentences:
            tokenized_sents = [word_tokenize(i) for i in sentence]
            for token in tokenized_sents:
                for each in token:
                    lower_text = each.lower()
                    punc = string.punctuation
                    punc = punc.replace('-','')
                    punc = punc.replace('\'','')
                    punc = list(punc)
                    for p in punc:
                        lower_text = lower_text.replace(p,' ')
                    lower_text = lower_text.replace('-',' ')
                    words.append(lower_text)
        return words

## get words
train_words= get_words(file_loc='train.pair_tok.tsv')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jvret\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


TypeError: 'decode' is an invalid keyword argument for open()

In [11]:
from collections import Counter, defaultdict
from tqdm import tqdm
vocab_size = 10000

def explode_word(word):
    w = "#" + word + "#"
    for i in range(0, len(w)-3+1):
        yield w[i:i+3]

count = Counter()
for w in tqdm(train_words):
    count.update(explode_word(w))

count = [["###", -1]] + count.most_common(vocab_size - 1)
dictionary = defaultdict(lambda: 0)

for i, k in enumerate(count):
    dictionary[k[0]] = i

known_trigrams = set(dictionary.keys())
print(known_trigrams)

100%|████████████████████████████████████████████████████████████████████████████████████████| 55143/55143 [00:00<00:00, 161330.19it/s]

{'sod', 'ify', 'ag#', 'rdo', 'eve', 'pie', 'amd', 'ndf', 'ous', 'rew', 'lk#', 'sel', 'udy', 'sid', 'gui', 'eeb', 'ht#', 'los', 'sai', 'ick', 'ezo', 'mus', 'wel', 'nuc', 'lag', 'rif', 'uin', 'ma#', '#ef', 'wee', '#gh', 'wir', 'hot', 'ede', 'rio', 'mum', 'alb', 'agm', 'eet', 'tau', '#zi', 'you', 'jay', 'onb', 'wen', 'ths', 'yin', 'ars', 'id#', 'tco', 'hon', 'blo', 'lsv', 'eit', 'aci', 'sym', '#me', 'exa', 'cco', 'elt', 'lum', 'fir', 'hat', 'pio', 'asu', '98#', '#lo', '#gu', 'osc', 'nia', 'ubl', 'pai', 'did', '#eg', 'ool', 'inl', 'dfh', 'sui', 'cab', 'hoo', '#co', 'ey#', 'ege', 'onm', 'gn#', 'mag', 'tax', 'nci', '3th', 'teo', 'dru', 'ueg', 'eam', 'rta', 'plo', 'oym', '#ep', 'omy', 'ape', '08#', 'oz#', 'urk', 'anu', 'ary', 'lke', 'opt', 'oy#', 'mat', 'pti', 'mbo', 'dol', '#te', 'nob', '#cy', 'quo', 'zoo', 'gmt', '#at', 'led', 'its', 'til', 'arp', 'rcu', 'ish', 'xch', 'inj', '###', 'uma', 'xce', 'dje', 'iel', 'bbe', 'jec', 'yde', 'rne', '#ly', '#gm', 'het', 'gh#', 'sol', '#ph', 'cad', '00s'

In [14]:
dictionary = defaultdict(lambda: 0)

def word2tri(word):
    w = "#" + word + "#"
    for i in range(0, len(w)-3+1):
        yield dictionary[w[i:i+3]]

def generate_batch():
    global data_index
    buffer_x = []
    buffer_y = []
    for _ in range(words_per_batch):
        x = [0] * vocab_size
        for i in word2tri(train_words[data_index + half_window]):
            x[i] = 1
        for _ in range(num_skips):
            y = [0] * positive_samples
            sample_word = rng.randint(-half_window, half_window - 1)
            if half_window >= 0: 
                sample_word += 1 
            sample_tris = list(word2tri(train_words[data_index + half_window + sample_word]))
            for i, k in enumerate(rng.choice(sample_tris, positive_samples)):
                y[i] = k
            buffer_x.append(x)
            buffer_y.append(y)
        data_index = (data_index + 1) % (vocab_size - window_size)
    return buffer_x, buffer_y
## generate X, Y batch
X, Y = generate_batch()

In [ ]:
np.set_printoptions(precision=3)
with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for epoch in range(training_epochs):
        cc = []
        for _ in range(round((vocab_size - window_size) / words_per_batch)):
            batch_x, batch_y = generate_batch()
            _, c = sess.run([optimizer, cost], feed_dict={layer_input: batch_x, layer_output_nums: batch_y})
            cc.append(c)
        print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(np.mean(np.array(cc))))

Epoch: 0001 cost= 532.805236816
Epoch: 0002 cost= 244.998672485
Epoch: 0003 cost= 140.980514526
Epoch: 0004 cost= 85.367652893
Epoch: 0005 cost= 53.230663300
Epoch: 0006 cost= 35.176074982
Epoch: 0007 cost= 24.562246323
Epoch: 0008 cost= 17.946758270
Epoch: 0009 cost= 14.207668304
Epoch: 0010 cost= 11.434528351
Epoch: 0011 cost= 9.431775093
Epoch: 0012 cost= 8.346570015
Epoch: 0013 cost= 7.402019024
Epoch: 0014 cost= 6.698512077
